## 1. Configuration

In [ ]:
# Configuration
NUM_AGENTS = 3
STREAM_NAME = "agent_stream"
REDIS_HOST = "localhost"
REDIS_PORT = 6379
RUN_DURATION_SECONDS = 60  # How long to run the demo

print(f"Configuration: {NUM_AGENTS} agents, stream='{STREAM_NAME}', duration={RUN_DURATION_SECONDS}s")

## 2. Imports

In [ ]:
import asyncio
import nest_asyncio

# Enable nested event loops for Jupyter
nest_asyncio.apply()

from agents.network_agent import NetworkAgent
from agents.prompt_configs.generate_prompt import PromptGenerator
from controller.time_manager import TimeManager
from controller.order_manager import OrderManager
from network.cache import RedisCache
from logs.logger import Logger, console_logger

print("Imports successful!")

## 3. Initialize Shared Components

In [ ]:
# Initialize PromptGenerator with shared topic
prompt_generator = PromptGenerator()
topic = prompt_generator.get_topic()
print(f"Shared discussion topic: {topic}")

# Generate unique prompts for each agent
agent_prompts = prompt_generator.generate_multiple_prompts(NUM_AGENTS)
for i, prompt in enumerate(agent_prompts):
    print(f"Agent {i+1} prompt: {prompt[:80]}...")

In [ ]:
# Initialize TimeManager with 3-second global interval
time_manager = TimeManager(global_interval=3.0)
print(f"TimeManager initialized with {time_manager.global_interval}s global interval")

# Initialize RedisCache for storing agent message histories
message_cache = RedisCache(host=REDIS_HOST, port=REDIS_PORT)
print(f"RedisCache connected to {REDIS_HOST}:{REDIS_PORT}")

# Initialize Logger for recording agent publishes
logger = Logger(num_agents=NUM_AGENTS)
print(f"Logger initialized, writing to: {logger.file_path}")

## 4. Create Agents

In [ ]:
# Create agents with unique prompts but shared topic
agents = []
for i in range(NUM_AGENTS):
    agent_id = f"agent_{i+1}"
    init_prompt = (
        f"You are agent {i+1} in a multi-agent discussion about '{topic}'. "
        f"{agent_prompts[i]} "
        "Respond thoughtfully and concisely, keeping the conversation going."
    )
    agent = NetworkAgent(
        id=agent_id,
        init_prompt=init_prompt,
        stream_name=STREAM_NAME,
        stream_group=f"group_{i+1}",
        redis_host=REDIS_HOST,
        redis_port=REDIS_PORT,
        time_manager=time_manager,
        order_manager=None,  # Will set after OrderManager is created
        message_cache=message_cache,
        logger=logger,
    )
    agents.append(agent)
    print(f"Created {agent_id}")

print(f"\nTotal agents created: {len(agents)}")

In [ ]:
# Initialize OrderManager with all agents
order_manager = OrderManager(
    agents=agents,
    message_cache=message_cache,
    redis_host=REDIS_HOST,
    redis_port=REDIS_PORT,
)
print("OrderManager initialized")

# Inject OrderManager into each agent
for agent in agents:
    agent.order_manager = order_manager
print("OrderManager injected into all agents")

## 5. Start Agents and Run Conversation

In [ ]:
async def start_agents():
    """Start all agents."""
    print("Starting agents...")
    for agent in agents:
        await agent.start()
    print(f"All {NUM_AGENTS} agents are now running!")

# Start agents
asyncio.get_event_loop().run_until_complete(start_agents())

In [ ]:
async def kick_off_conversation():
    """Start the conversation with an initial message."""
    initial_message = (
        f"Let's begin our discussion about {topic}. "
        "What are your initial thoughts on this subject?"
    )
    print(f"Agent 1 starting conversation: {initial_message}")
    await agents[0].publish_message(initial_message)
    print("Initial message published!")

# Kick off conversation
asyncio.get_event_loop().run_until_complete(kick_off_conversation())

In [ ]:
async def run_conversation(duration: int):
    """Let the conversation run for the specified duration."""
    print(f"Running conversation for {duration} seconds...")
    await asyncio.sleep(duration)
    print("Conversation complete!")

# Run the conversation
asyncio.get_event_loop().run_until_complete(run_conversation(RUN_DURATION_SECONDS))

## 6. Cleanup

In [ ]:
async def cleanup():
    """Clean up resources."""
    print("Shutting down...")
    await logger.async_stop()
    await message_cache.close()
    print("Done!")

# Cleanup
asyncio.get_event_loop().run_until_complete(cleanup())

## 7. Inspect Results (Optional)

After running the conversation, you can inspect the logged messages and cached data.

In [ ]:
# Read the log file
with open(logger.file_path, 'r') as f:
    logs = f.read()
print("=== Agent Publish Logs ===")
print(logs)

In [ ]:
async def inspect_cache():
    """Inspect cached messages for each agent."""
    # Reconnect cache for inspection
    cache = RedisCache(host=REDIS_HOST, port=REDIS_PORT)
    print("=== Cached Messages per Agent ===")
    for agent in agents:
        messages = await cache.get_responses(agent.id, last_n=5)
        print(f"\n{agent.id} (last 5 messages):")
        for msg in messages:
            print(f"  - {msg}")
    await cache.close()

asyncio.get_event_loop().run_until_complete(inspect_cache())